In [ ]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

In [0]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

In [ ]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

# 项目1-PM2.5预测

## 友情提示
同学们可以前往课程作业区先行动手尝试！！！

## 项目描述
* 本次作业的资料是从行政院环境环保署空气品质监测网所下载的观测资料。
* 希望大家能在本作业实现 linear regression 预测出 PM2.5 的数值。

## 数据集介绍
* 本次作业使用丰原站的观测记录，分成 train set 跟 test set，train set 是丰原站每个月的前 20 天所有资料。test set 则是从丰原站剩下的资料中取样出来。
* train.csv: 2014年每个月前 20 天的完整资料。
* test.csv : 从剩下的资料当中取样出连续的 10 小时为一笔，<u>**前九小时的所有观测数据**</u>当作 feature，第十小时的 PM2.5 当作 answer。一共取出 240 笔不重複的 test data，请根据 feature 预测这 240 笔的 PM2.5。
* Data 含有 18 项观测数据 AMB_TEMP, CH4, CO, NHMC, NO, NO2, NOx, O3, PM10, PM2.5, RAINFALL, RH, SO2, THC, WD_HR, WIND_DIREC, WIND_SPEED, WS_HR。  

## 项目要求
- 请手动实现 linear regression，方法限使用 gradient descent。
- 禁止使用 numpy.linalg.lstsq

## 数据准备
无

## 环境配置/安装

In [1]:
!pip install --upgrade pandas

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 9.9MB 30.5MB/s eta 0:00:01
     |████████████████████████████████| 15.7MB 7.6MB/s eta 0:00:011
ERROR: blackhole 1.0.1 has requirement numpy<=1.19.5, but you'll have numpy 1.21.0 which is incompatible.
ERROR: blackhole 1.0.1 has requirement pandas<=1.1.5,>=0.24.0, but you'll have pandas 1.2.5 which is incompatible.
  Found existing installation: numpy 1.16.4
    Uninstalling numpy-1.16.4:
      Successfully uninstalled numpy-1.16.4
  Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5


In [41]:
import sys
import pandas as pd
import numpy as np
data = pd.read_csv('work/hw1_data/train.csv',encoding='big5')
data

,日期,測站,測項,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
0,2014/1/1,豐原,AMB_TEMP,14,14,14,13,12,12,12,...,22,22,21,19,17,16,15,15,15,15
1,2014/1/1,豐原,CH4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,2014/1/1,豐原,CO,0.51,0.41,0.39,0.37,0.35,0.3,0.37,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,2014/1/1,豐原,NMHC,0.2,0.15,0.13,0.12,0.11,0.06,0.1,...,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,2014/1/1,豐原,NO,0.9,0.6,0.5,1.7,1.8,1.5,1.9,...,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,2014/12/20,豐原,THC,1.8,1.8,1.8,1.8,1.8,1.7,1.7,...,1.8,1.8,2,2.1,2,1.9,1.9,1.9,2,2
4316,2014/12/20,豐原,WD_HR,46,13,61,44,55,68,66,...,59,308,327,21,100,109,108,114,108,109
4317,2014/12/20,豐原,WIND_DIREC,36,55,72,327,74,52,59,...,18,311,52,54,121,97,107,118,100,105
4318,2014/12/20,豐原,WIND_SPEED,1.9,2.4,1.9,2.8,2.3,1.9,2.1,...,2.3,2.6,1.3,1,1.5,1,1.7,1.5,2,2


# **预处理** 
取需要的数值部分，将 'RAINFALL' 栏位全部补 0。

In [42]:
data = data.iloc[:,3:]
data[data == 'NR'] = 0
raw_data = data.to_numpy()
print(raw_data.shape)

(4320, 24)


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/pandas/core/frame.py:3215: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


In [43]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,14,14,14,13,12,12,12,12,15,17,...,22,22,21,19,17,16,15,15,15,15
1,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,0.51,0.41,0.39,0.37,0.35,0.3,0.37,0.47,0.78,0.74,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,0.2,0.15,0.13,0.12,0.11,0.06,0.1,0.13,0.26,0.23,...,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,0.9,0.6,0.5,1.7,1.8,1.5,1.9,2.2,6.6,7.9,...,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5


# **提取特征 (1)**
![图片说明](https://ai-studio-static-online.cdn.bcebos.com/8dae829615d54e8f8a002bc59330734e24ae4fd8d30c48249c9fd89d4c8452b3)
![图片说明](https://ai-studio-static-online.cdn.bcebos.com/271f6ec86c794658b92cb93156b5674d746426dfd699444db0c8aa340f595caf)

将原始 4320 * 18 的资料依照每个月份重组成 12 个 18 (特征) * 480 (小时) 的资料。 

In [44]:
month_data = {}
for month in range(12):
    sample = np.empty([18,480])
    for day in range(20):
        sample[:, day * 24 : (day + 1) * 24] = raw_data[18 * (20 * month + day) : 18 * (20 * month + day + 1), :]
    month_data[month] = sample
    

In [45]:
print(month_data.keys())
print(len(month_data))
#print(month_data.values())
print(month_data[0].shape)

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
12
(18, 480)


# **提取特征 (2)**
![alt text](https://ai-studio-static-online.cdn.bcebos.com/b331ac245ba04847b299b3fbf00208ee66ba7e922d354a9a8d2eb223491d4f05)
![alt text](https://ai-studio-static-online.cdn.bcebos.com/aa9319162fda4ab5bc989211167d853a93f63173efb5482bab4cf206b764f7e1)

每个月会有 480小时，每 9 小时形成一个数据，每个月会有 471 个数据，故总资料数为 471 * 12 笔，而每笔 数据 有 9 * 18 的 特征 (一小时 18 个 特征 * 9 小时)。

对应的 目标 则有 471 * 12 个(第 10 个小时的 PM2.5)

In [46]:
x = np.empty([12 * 471, 18 * 9], dtype = float)
y = np.empty([12 * 471, 1], dtype = float)
for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day == 19 and hour > 14:
                continue
            x[month * 471 + day * 24 + hour, :] = month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1, -1)
            #vector dim: 18*9 (9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9)
            y[month * 471 + day * 24 + hour, 0] = month_data[month][9, day * 24 + hour + 9] # value
print(x)
print(y)

[[14.  14.  14.  ...  2.   2.   0.5]
 [14.  14.  13.  ...  2.   0.5  0.3]
 [14.  13.  12.  ...  0.5  0.3  0.8]
 ...
 [17.  18.  19.  ...  1.1  1.4  1.3]
 [18.  19.  18.  ...  1.4  1.3  1.6]
 [19.  18.  17.  ...  1.3  1.6  1.8]]
[[30.]
 [41.]
 [44.]
 ...
 [17.]
 [24.]
 [29.]]


In [47]:
print(x.shape)
print(y.shape)

(5652, 162)
(5652, 1)


# **归一化**

In [48]:
mean_x = np.mean(x, axis = 0) #18 * 9
std_x = np.std(x, axis = 0) # 18 * 9
for i in range(len(x)): #12 * 471
    for j in range(len(x[0])): # 18 * 8
        if std_x[j] != 0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]
print(x)

[[-1.35825331 -1.35883937 -1.359222   ...  0.26650729  0.2656797
  -1.14082131]
 [-1.35825331 -1.35883937 -1.51819928 ...  0.26650729 -1.13963133
  -1.32832904]
 [-1.35825331 -1.51789368 -1.67717656 ... -1.13923451 -1.32700613
  -0.85955971]
 ...
 [-0.88092053 -0.72262212 -0.56433559 ... -0.57693779 -0.29644471
  -0.39079039]
 [-0.7218096  -0.56356781 -0.72331287 ... -0.29578943 -0.39013211
  -0.1095288 ]
 [-0.56269867 -0.72262212 -0.88229015 ... -0.38950555 -0.10906991
   0.07797893]]


# **将训练数据分割成 "训练集 "和 "验证集"**
这部分是简单示范，以生成比较中用来训练的训练集和不会被放入训练、只是用来验证的验证集。

In [49]:
import math
x_train_set = x[: math.floor(len(x) * 0.8), :]
y_train_set = y[: math.floor(len(x) * 0.8), :]
x_validation = x[math.floor(len(x) * 0.8):, :]
y_validation = y[math.floor(len(x) * 0.8):, :]
print(x_train_set)
print(y_train_set)
print(x_validation)
print(y_validation)
print(len(x_train_set))
print(len(y_train_set))
print(len(x_validation))
print(len(y_validation))

[[-1.35825331 -1.35883937 -1.359222   ...  0.26650729  0.2656797
  -1.14082131]
 [-1.35825331 -1.35883937 -1.51819928 ...  0.26650729 -1.13963133
  -1.32832904]
 [-1.35825331 -1.51789368 -1.67717656 ... -1.13923451 -1.32700613
  -0.85955971]
 ...
 [ 0.86929969  0.70886668  0.38952809 ...  1.39110073  0.2656797
  -0.39079039]
 [ 0.71018876  0.39075806  0.07157353 ...  0.26650729 -0.39013211
  -0.39079039]
 [ 0.3919669   0.07264944  0.07157353 ... -0.38950555 -0.39013211
  -0.85955971]]
[[30.]
 [41.]
 [44.]
 ...
 [ 7.]
 [ 5.]
 [14.]]
[[ 0.07374504  0.07264944  0.07157353 ... -0.38950555 -0.85856912
  -0.57829812]
 [ 0.07374504  0.07264944  0.23055081 ... -0.85808615 -0.57750692
   0.54674825]
 [ 0.07374504  0.23170375  0.23055081 ... -0.57693779  0.54674191
  -0.1095288 ]
 ...
 [-0.88092053 -0.72262212 -0.56433559 ... -0.57693779 -0.29644471
  -0.39079039]
 [-0.7218096  -0.56356781 -0.72331287 ... -0.29578943 -0.39013211
  -0.1095288 ]
 [-0.56269867 -0.72262212 -0.88229015 ... -0.3895055

# **训练**
![alt text](https://ai-studio-static-online.cdn.bcebos.com/1a34083d8bde48b5b0e09248e38fbd8663509f79b79a48d7bc23d0e32941ebed)
![alt text](https://ai-studio-static-online.cdn.bcebos.com/a453c7b4d72444fc92dfde3efd96b1b9d3f7b81f7d3f4da7a870e9b1aba558e1)
![alt text](https://ai-studio-static-online.cdn.bcebos.com/bd3836fbb0fe4674933463d56e5be5f0436c8e5e62b044b9a7ac8aeac714054c)

(和上图不同处: 下面的代码采用均方根误差)

因为常数项的存在，所以 维度 (dim) 需要多加一栏；eps 项是避免 adagrad 的分母为 0 而加的极小数值。

每一个 维度 (dim) 会对应到各自的梯度, 权重 (w)，透过一次次的迭代 (iter_time) 学习。

**Implement linear regression**  
<u>Pseudo code</u> 
1. declare weight vector, initial lr, and # of iteration  
2. for i_th iteration:  
3. y'= the inner product of train_x and weight vector  
4. $Loss = y' - train_y$    
5. $gradient = 2 * np.dot((train_x)',L)$  
6. weight vector -= learning rate * gradient  
**3**   
$$
\begin{pmatrix}y_{1}^{\prime}
 \\y_{2}^{\prime} 
 \\\vdots 
 \\y_{n}^{\prime}

\end{pmatrix}=\begin{pmatrix}
 x_{11}^{} &  \dots  & x_{1p}^{}\\
  x_{21}^{} &  \dots  & x_{2p}^{}\\
\vdots  &  \ddots   & \vdots\\
x_{n1}^{} &  \dots  & x_{np}^{}\\
\end{pmatrix}\cdot \begin{pmatrix}w_{1}^{}\\
 w_{2}^{}\\
 \vdots \\
 w_{3}^{}\\

\end{pmatrix}
$$  
**4**  
$$
L = \begin{pmatrix}y_{1}^{\prime }
 \\y_{2}^{\prime} 
 \\\vdots 
 \\y_{n}^{\prime}
\end{pmatrix}-
\begin{pmatrix}y_{1}^{ }
 \\y_{2}^{} 
 \\\vdots 
 \\y_{n}^{}
\end{pmatrix}
$$
**5**  
**<center>gradient = 2 x</center>**

$$
\begin{pmatrix}
 x_{11}^{} & x_{21}^{} & \dots  & x_{n1}^{}\\
  x_{12}^{} &  x_{22}^{} &\dots  & x_{22}^{}\\
\vdots  & \vdots & \ddots   & \vdots\\
x_{1p}^{}&x_{2p}^{}&\dots  & x_{np}^{}\\
\end{pmatrix}
\begin{pmatrix}
 y_{1}^{\prime }-y_{1}\\
 y_{2}^{\prime }-y_{2}\\
\vdots \\
y_{n}^{\prime }-y_{n}\\
\end{pmatrix}
$$
**<center>gradient (p-dim vector)</center>**

**Adagrad**  
<u>Pseudo code</u>  
1. Declare weight vector, initial lr, and # of iteration  
**Decalre prev_gra storing gradients in every previous iterations**  
2. for i_th iteration:  
3. y'= the inner product of train_x and weight vector  
4. $Loss = y' - train_y$    
5. $gradient = 2 * np.dot((train_x)',L)$  
**prev_gra += gra^2**  
**ada = np.sqrt(prev_gra)**
6. weight vector -= learning rate * gradient/**ada**   


In [50]:
dim = 18 * 9 + 1
w = np.zeros([dim, 1])
x = np.concatenate((np.ones([12 * 471, 1]), x), axis = 1).astype(float)
learning_rate = 100
iter_time = 1000
adagrad = np.zeros([dim, 1])
print(adagrad.shape)
print('==========')
eps = 0.0000000001
for t in range(iter_time):
    loss = np.sqrt(np.sum(np.power(np.dot(x,w) - y, 2))/471/12)#rmse均方根误差
    if(t%100==0): 
        print(str(t) + ":" + str(loss))
    gradient = 2 * np.dot(x.transpose(), np.dot(x, w) -y)#dim*1
    #print(gradient.shape)
    adagrad += gradient ** 2
    w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
np.save('work/weight.npy',w)
print('===========')
print(gradient.shape)
print(adagrad.shape)
print('=============')
print(w.shape)
print(w)

(163, 1)
0:27.071214829194115
100:33.78905859777454
200:19.913751298197095
300:13.531068193689693
400:10.645466158446172
500:9.277353455475065
600:8.518042045956502
700:8.014061987588425
800:7.636756824775692
900:7.336563740371125
(163, 1)
(163, 1)
(163, 1)
[[ 2.13740269e+01]
 [ 3.58888909e+00]
 [ 4.56386323e+00]
 [ 2.16307023e+00]
 [-6.58545223e+00]
 [-3.38885580e+01]
 [ 3.22235518e+01]
 [ 3.49340354e+00]
 [-4.60308671e+00]
 [-1.02374754e+00]
 [-3.96791501e-01]
 [-1.06908800e-01]
 [ 2.22488184e-01]
 [ 8.99634117e-02]
 [ 1.31243105e-01]
 [ 2.15894989e-02]
 [-1.52867263e-01]
 [ 4.54087776e-02]
 [ 5.20999235e-01]
 [ 1.60824213e-01]
 [-3.17709451e-02]
 [ 1.28529025e-02]
 [-1.76839437e-01]
 [ 1.71241371e-01]
 [-1.31190032e-01]
 [-3.51614451e-02]
 [ 1.00826192e-01]
 [ 3.45018257e-01]
 [ 4.00130315e-02]
 [ 2.54331382e-02]
 [-5.04425219e-01]
 [ 3.71483018e-01]
 [ 8.46357671e-01]
 [-8.11920428e-01]
 [-8.00217575e-02]
 [ 1.52737711e-01]
 [ 2.64915130e-01]
 [-5.19860416e-02]
 [-2.51988315e-01]
 

In [51]:
print(w.shape)

(163, 1)


# **测试**
![alt text](https://ai-studio-static-online.cdn.bcebos.com/fc702c460628481889782e6a8f6490ce3e892c780e68481dbbb426dc9f459b31)

载入测试数据，并且以相似于训练资料预先处理和特徵萃取的方式处理，使测试数据形成 240 个维度为 18 * 9 + 1 的资料。

In [53]:
testdata = pd.read_csv('work/hw1_data/test.csv', header = None, encoding= 'big5')
test_data = testdata.iloc[:,2:]
test_data[test_data == 'NR'] = 0
test_data = test_data.to_numpy()
test_x = np.empty([240, 18 * 9], dtype = float)
for i in range(240):
    test_x[i,:] = test_data[18 * i : 18 * (i + 1), :].reshape(1,-1)
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        if std_x[j] != 0:
            test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]
test_x = np.concatenate((np.ones([240,1]), test_x), axis = 1).astype(float)
print(test_x)


[[ 1.         -0.24447681 -0.24545919 ... -0.67065391 -1.04594393
   0.07797893]
 [ 1.         -1.35825331 -1.51789368 ...  0.17279117 -0.10906991
  -0.48454426]
 [ 1.          1.5057434   1.34508393 ... -1.32666675 -1.04594393
  -0.57829812]
 ...
 [ 1.          0.3919669   0.54981237 ...  0.26650729 -0.20275731
   1.20302531]
 [ 1.         -1.8355861  -1.8360023  ... -1.04551839 -1.13963133
  -1.14082131]
 [ 1.         -1.35825331 -1.35883937 ...  2.98427476  3.26367657
   1.76554849]]


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/pandas/core/frame.py:3215: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


In [54]:
print(test_x.shape)

(240, 163)


# **预测**
说明图同上

![alt text](https://ai-studio-static-online.cdn.bcebos.com/fc702c460628481889782e6a8f6490ce3e892c780e68481dbbb426dc9f459b31)

有了 权重 和测试资料即可预测 目标。

In [55]:
w = np.load('work/weight.npy')
ans_y = np.dot(test_x, w)
print(ans_y)

[[ 5.17496040e+00]
 [ 1.83062143e+01]
 [ 2.04912181e+01]
 [ 1.15239429e+01]
 [ 2.66160568e+01]
 [ 2.05313481e+01]
 [ 2.19065510e+01]
 [ 3.17364687e+01]
 [ 1.33916741e+01]
 [ 6.44564665e+01]
 [ 2.02645688e+01]
 [ 1.53585761e+01]
 [ 6.85894728e+01]
 [ 4.84281137e+01]
 [ 1.87023338e+01]
 [ 1.01885957e+01]
 [ 3.07403629e+01]
 [ 7.11322178e+01]
 [-4.13051739e+00]
 [ 1.82356940e+01]
 [ 3.85789223e+01]
 [ 7.13115197e+01]
 [ 7.41034816e+00]
 [ 1.87179553e+01]
 [ 1.49372503e+01]
 [ 3.67197367e+01]
 [ 1.79616970e+01]
 [ 7.57894629e+01]
 [ 1.23093102e+01]
 [ 5.62953517e+01]
 [ 2.51131609e+01]
 [ 4.61024867e+00]
 [ 2.48377055e+00]
 [ 2.47594223e+01]
 [ 3.04802805e+01]
 [ 3.84639307e+01]
 [ 4.42023106e+01]
 [ 3.00868360e+01]
 [ 4.04736750e+01]
 [ 2.92264799e+01]
 [ 5.60645605e+00]
 [ 3.86660161e+01]
 [ 3.46102134e+01]
 [ 4.83896975e+01]
 [ 1.47572477e+01]
 [ 3.44668201e+01]
 [ 2.74831069e+01]
 [ 1.20008794e+01]
 [ 2.13780362e+01]
 [ 2.85444031e+01]
 [ 2.01655138e+01]
 [ 1.07966781e+01]
 [ 2.2171035

In [56]:
print(ans_y.shape)

(240, 1)


In [57]:
import csv
with open('work/submit.csv',mode='w',newline='') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['id','value']
    print(header)
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id_' + str(i), ans_y[i][0]]
        csv_writer.writerow(row)
        print(row)

['id', 'value']
['id_0', 5.17496039898474]
['id_1', 18.30621425352791]
['id_2', 20.491218094180528]
['id_3', 11.523942869805328]
['id_4', 26.616056752306154]
['id_5', 20.531348081761223]
['id_6', 21.90655101879737]
['id_7', 31.736468747068837]
['id_8', 13.391674055111707]
['id_9', 64.45646650291957]
['id_10', 20.264568836159455]
['id_11', 15.358576077361239]
['id_12', 68.58947276926726]
['id_13', 48.42811374745719]
['id_14', 18.70233382419321]
['id_15', 10.188595737466713]
['id_16', 30.740362859820426]
['id_17', 71.13221776355108]
['id_18', -4.130517391262437]
['id_19', 18.2356940164287]
['id_20', 38.57892227500776]
['id_21', 71.31151972531332]
['id_22', 7.410348162634065]
['id_23', 18.71795533032141]
['id_24', 14.937250260084582]
['id_25', 36.71973669470534]
['id_26', 17.961697005662707]
['id_27', 75.78946287210536]
['id_28', 12.309310248614468]
['id_29', 56.295351739649604]
['id_30', 25.113160865661484]
['id_31', 4.610248674094032]
['id_32', 2.48377055451501]
['id_33', 24.75942226132

In [58]:
w = np.load('work/weight.npy')
print(w.shape)
print(w)

(163, 1)
[[ 2.13740269e+01]
 [ 3.58888909e+00]
 [ 4.56386323e+00]
 [ 2.16307023e+00]
 [-6.58545223e+00]
 [-3.38885580e+01]
 [ 3.22235518e+01]
 [ 3.49340354e+00]
 [-4.60308671e+00]
 [-1.02374754e+00]
 [-3.96791501e-01]
 [-1.06908800e-01]
 [ 2.22488184e-01]
 [ 8.99634117e-02]
 [ 1.31243105e-01]
 [ 2.15894989e-02]
 [-1.52867263e-01]
 [ 4.54087776e-02]
 [ 5.20999235e-01]
 [ 1.60824213e-01]
 [-3.17709451e-02]
 [ 1.28529025e-02]
 [-1.76839437e-01]
 [ 1.71241371e-01]
 [-1.31190032e-01]
 [-3.51614451e-02]
 [ 1.00826192e-01]
 [ 3.45018257e-01]
 [ 4.00130315e-02]
 [ 2.54331382e-02]
 [-5.04425219e-01]
 [ 3.71483018e-01]
 [ 8.46357671e-01]
 [-8.11920428e-01]
 [-8.00217575e-02]
 [ 1.52737711e-01]
 [ 2.64915130e-01]
 [-5.19860416e-02]
 [-2.51988315e-01]
 [ 3.85246517e-01]
 [ 1.65431451e-01]
 [-7.83633314e-02]
 [-2.89457231e-01]
 [ 1.77615023e-01]
 [ 3.22506948e-01]
 [-4.59955256e-01]
 [-3.48635358e-02]
 [-5.81764363e-01]
 [-6.43394528e-02]
 [-6.32876949e-01]
 [ 6.36624507e-02]
 [ 8.31592506e-02]
 [-

以上打印的部分主要是为了看一下资料和结果的呈现，拿掉也无妨。另外，在自己的 linux 系统，可以将档案写死的的部分换成 **sys.argv** 的使用 (可在 终端 自行输入档案和档案位置)。
最后，可以藉由调整 (learning_rate)学习率、iter_time (迭代次数)、取用特征的多少(取几个小时，取哪些特征栏位)，甚至是不同的 模型来超越基线。